In [225]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [226]:
experiments = raw_input("experiments ") 
experiments = experiments.split(",")

experiments 1,2


In [227]:
experiments

['1', '2']

In [229]:
training = 1
testing = 2

dirs = []
for exp in experiments:
    print exp
    experiment_root = "processed/experiment_" + str(exp) + "/"
    sub_dir = [x[0] for x in os.walk(experiment_root)]
    dirs.extend(sub_dir)
    #mfcc_root = experiment_root +

In [208]:
experiment_root = "processed/experiment_" 

In [209]:
def read_data(env, experiment_root):
    features = pd.DataFrame()
    files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(experiment_root + env):
        for file in f:
            if 'all_mots' in file:
                files.append(os.path.join(r, file))

    for f in files:
        print(f)
        rd = pd.read_csv(f,header=None)
        features = pd.concat([features,rd],axis=0)
    return features

In [210]:
for env in experiments:
    if env == str(training):
        training_data = read_data(env, experiment_root)
    if env == str(testing):
        testing_data = read_data(env, experiment_root)

processed/experiment_1/5/training_phase/all_mots2
processed/experiment_1/4/training_phase/all_mots2
processed/experiment_1/3/training_phase/all_mots2
processed/experiment_1/2/training_phase/all_mots2
processed/experiment_1/1/training_phase/all_mots2
processed/experiment_2/10/training_phase/all_mots2
processed/experiment_2/9/training_phase/all_mots2
processed/experiment_2/4/training_phase/all_mots2
processed/experiment_2/8/training_phase/all_mots2
processed/experiment_2/7/training_phase/all_mots2


In [211]:
len(training_data)

186

In [212]:
len(testing_data)

183

# Class balancing (TBA)

In [213]:
training_data[0].value_counts()

other      163
bedroom     23
Name: 0, dtype: int64

In [214]:
def random_undersampling(mots):
    #mots = mots.reset_index()
    reduced_mots = pd.DataFrame()
    labels = mots[0].values.reshape(-1)
    feature_num = np.unique(labels, return_counts=True)
    print feature_num
    min_feature_num = min(feature_num[1])
    print min_feature_num
    feature_num = np.stack(feature_num, axis=1) # label of each cluster and number of instances
    
    for x in feature_num:
        delete_itr = len(mots.loc[mots[0] ==  x[0]]) - min_feature_num
        label_loc = all_mots.loc[mots[0] == x[0]]
        randoms = label_loc.sample(delete_itr)
        display(randoms.head())
        mots.drop(randoms.index,inplace = True)
        #mots = mots.drop([mots.loc[mots.index == x[0]].sample(delete_itr)], axis = 0)
        #mots = mots.drop(mots.index[mots.loc[mots.index ==  x[0]].sample(delete_itr).index.values])
        #mots = mots.drop(all_mots.loc[[mots.loc[mots[0] ==  x[0]].sample(delete_itr).index.values]].index.vlues,axis=0)
        #mots = mots.reset_index(drop=True)
    return mots

In [215]:
from sklearn.utils import resample
def random_upsampling(mots):
    reduced_mots = pd.DataFrame()
    labels = mots[0].values.reshape(-1)
    feature_num = np.unique(labels, return_counts=True)
    print feature_num
    max_feature_num = max(feature_num[1])
    print max_feature_num
    feature_num = np.stack(feature_num, axis=1) # label of each cluster and number of instances
    
    for x in feature_num:
        if x[0] != "other":
            display(mots.loc[mots[0]==x[0]])

            up = resample(mots.loc[mots[0]==x[0]],replace=True,n_samples=max_feature_num,random_state=27) # reproducible results

            mots = pd.concat([mots, up])
        
    return mots

In [216]:
training_data = random_upsampling(training_data)

(array(['bedroom', 'other'], dtype=object), array([ 23, 163]))
163


,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
2,bedroom,-1.000002e-08,-9.999996e-09,-9.999956e-09,-9.999998e-09,-1.000001e-08,-9.999999e-09,-1.000001e-08,-1.000000e-08,-9.999999e-09,...,104.201109,40.525187,58.544997,84.958338,128.548620,108.577590,54.662519,14.887518,90.553921,71.826242
3,bedroom,-1.000000e-08,-1.000001e-08,-1.000002e-08,-1.000002e-08,-1.000000e-08,-1.000000e-08,-9.999992e-09,-1.000000e-08,-1.000000e-08,...,108.800858,35.274056,48.656022,69.172325,140.316133,137.211404,19.892915,14.776231,100.456162,66.023845
4,bedroom,-1.000001e-08,-9.999997e-09,-1.000001e-08,-9.999968e-09,-9.999998e-09,-1.000000e-08,-9.999991e-09,-1.000000e-08,-1.000000e-08,...,94.994534,32.236543,46.062244,82.053063,136.796752,115.433220,21.275304,15.715946,104.477363,58.040105
5,bedroom,-9.999994e-09,-1.000002e-08,-9.999986e-09,-9.999981e-09,-9.999997e-09,-1.000001e-08,-9.999994e-09,-1.000001e-08,-9.999994e-09,...,91.965407,31.766670,64.707015,92.129290,130.968242,95.597159,30.466287,17.380052,76.565961,98.830930
30,bedroom,-1.000001e-08,-1.000000e-08,-1.000001e-08,-9.999961e-09,-1.000001e-08,-9.999997e-09,-9.999987e-09,-1.000001e-08,-1.000001e-08,...,64.358670,41.302440,66.539962,170.362443,124.222865,22.798150,39.894503,129.634948,101.408895,40.476421
31,bedroom,-1.000001e-08,-9.999999e-09,-1.000002e-08,-1.000003e-08,-9.999994e-09,-1.000000e-08,-1.000000e-08,-9.999966e-09,-9.999993e-09,...,64.637347,42.891771,108.010730,174.267077,109.634870,25.151621,85.407878,129.941006,91.319880,25.334121
32,bedroom,-1.000002e-08,-1.000000e-08,-9.999950e-09,-1.000001e-08,-1.000001e-08,-1.000000e-08,-1.000000e-08,-1.000000e-08,-1.000000e-08,...,74.917197,40.998801,68.507474,172.972330,131.589129,20.786202,47.455669,130.672849,93.379044,51.941270
33,bedroom,-9.999989e-09,-1.000001e-08,-9.999999e-09,-9.999968e-09,-9.999990e-09,-1.000001e-08,-9.999989e-09,-1.000001e-08,-1.000000e-08,...,58.826395,37.820344,40.201297,156.468892,110.433979,20.971250,29.255524,118.481488,83.997482,51.778112
34,bedroom,-9.999977e-09,-9.999998e-09,-9.999979e-09,-1.000004e-08,-9.999991e-09,-9.999987e-09,-1.000000e-08,-9.999978e-09,-9.999999e-09,...,60.782683,40.219057,40.025303,155.225937,111.777918,21.765593,29.439503,117.651864,77.639627,56.609358
33,bedroom,-9.999997e-09,-9.999994e-09,-9.999958e-09,-9.999961e-09,-9.999996e-09,-9.999994e-09,-9.999998e-09,-1.000000e-08,-1.000000e-08,...,81.938767,37.081428,64.314993,119.570582,114.696152,65.781220,36.605736,51.151255,69.886780,67.649728


In [217]:
training_data[0].value_counts()

bedroom    186
other      163
Name: 0, dtype: int64

In [218]:
training_labels = training_data[0].values
training_data.drop([0], axis=1,inplace=True)

In [219]:
testing_labels = testing_data[0].values
testing_data.drop([0], axis=1,inplace=True)

# Making a random forest model and training (first layer)

In [220]:
from sklearn.ensemble import RandomForestClassifier
import pickle

def randomForest(train, train_labels):
    # Create the model with 100 trees
    model = RandomForestClassifier(n_estimators=100, 
                                   max_features = 'sqrt')
    # Fit on training data
    model.fit(train, train_labels)
    #pickle.dump(model, open(root + str(test_env) + "/training_phase/" + "randomForest.sav", 'wb'))
    return model 

model = randomForest(training_data, training_labels)#location_list)

In [221]:
predict = model.predict(testing_data)
predict_probs = model.predict_proba(testing_data)

In [222]:
predict

array(['other', 'bedroom', 'bedroom', 'other', 'other', 'other', 'other',
       'other', 'other', 'other', 'other', 'other', 'other', 'other',
       'other', 'other', 'other', 'other', 'other', 'other', 'other',
       'other', 'other', 'other', 'other', 'other', 'other', 'other',
       'other', 'other', 'other', 'other', 'other', 'other', 'other',
       'other', 'other', 'other', 'other', 'other', 'other', 'other',
       'other', 'other', 'other', 'other', 'other', 'other', 'other',
       'other', 'other', 'other', 'other', 'other', 'other', 'other',
       'other', 'other', 'other', 'other', 'other', 'other', 'other',
       'bedroom', 'other', 'bedroom', 'bedroom', 'bedroom', 'bedroom',
       'other', 'other', 'other', 'other', 'other', 'other', 'other',
       'other', 'other', 'other', 'other', 'other', 'other', 'other',
       'other', 'other', 'other', 'other', 'other', 'other', 'other',
       'other', 'other', 'other', 'other', 'other', 'other', 'other',
       'other',

In [223]:
from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(testing_labels, predict))  
print(classification_report(testing_labels, predict))

[[ 11   0   3]
 [  0   0   1]
 [  3   0 165]]
              precision    recall  f1-score   support

     bedroom       0.79      0.79      0.79        14
      office       0.00      0.00      0.00         1
       other       0.98      0.98      0.98       168

   micro avg       0.96      0.96      0.96       183
   macro avg       0.59      0.59      0.59       183
weighted avg       0.96      0.96      0.96       183



/home/thilina/.local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [224]:
from sklearn.metrics import confusion_matrix
CM = confusion_matrix(testing_labels, predict)

df_cm = pd.DataFrame(CM, index = [i for i in set(predict)],
                  columns = [i for i in set(predict)])


import seaborn
plt.figure(figsize=(12, 10))
seaborn.heatmap(df_cm,cmap='Blues',annot=True,fmt='g')
#plt.savefig(test_root+"confusion_mat.png")
plt.show()

ValueError: Shape of passed values is (3, 3), indices imply (2, 2)